# Importing libraries

In [1]:
import cassandra

# Connecting to db

In [2]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

# Creating  and connecting to keyspace

In [4]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [5]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Two question to be asked

1. Give me every album in my music library that was released in a given year

`select * from music_library WHERE YEAR=1970`

2. Give me every album in my music library that was created by a given artist

`select * from artist_library WHERE artist_name="The Beatles"`

### Because I want to do two different queries, I am going to do need different tables that partition the data differently.

Table Name: music_library<br>
column 1: Year<br>
column 2: Artist Name<br>
column 3: Album Name<br>
PRIMARY KEY(year, artist name)<br>
<br>
Table Name: album_library <br>
column 1: Artist Name<br>
column 2: Year<br>
column 3: Album Name<br>
PRIMARY KEY (artist name, year)<br>

# Creating two tables

In [8]:
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS album_library "
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY (artist_name,  year))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Inserting data in both tables

In [11]:
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

query1 = "INSERT INTO album_library (artist_name, year, album_name)"
query1 = query1 + " VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1966, "The Monkees", "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1970, "The Carpenters", "Close To You"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1970, "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1965, "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Who", 1965, "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Monkees", 1966, "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Carpenters", 1970, "Close To You"))
except Exception as e:
    print(e)

# Validating for query 1 using table 1

`select * from music_library WHERE YEAR=1970`

In [12]:
query = "select * from music_library WHERE YEAR=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.artist_name, row.album_name,)

1970 The Beatles Let it Be
1970 The Carpenters Close To You


# Validating for query 2 using table 2

`select * from album_library WHERE ARTIST_NAME = "The Beatles"`

In [13]:
query = "select * from album_library WHERE ARTIST_NAME='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.year, row.album_name)

The Beatles 1965 Rubber Soul
The Beatles 1970 Let it Be


# Dropping tables

In [14]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table album_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Closing connection

In [16]:
session.shutdown()
cluster.shutdown()